In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# pd.set_option('display.max_columns', 10)
# pd.set_option('display.max_rows', 100)


In [ ]:
ds_path = r".\data\GermanyUsedCars.zip"
dfo = pd.read_csv(ds_path, index_col=0, dtype_backend='pyarrow')
# dfo = pd.read_csv(ds_path, index_col=0)
dfo.columns

### Data exploration

In [ ]:
# dfo.shape
# dfo.isnull().count()
# dfo.isna().sum()
# dfo.isnull().sum()
# dfo.fuel_type.value_counts()
# dfo.color.value_counts()
# dfo.color.isna().sum()
# df.dtypes
# dfo.mileage_in_km.value_counts()
# dfo.mileage_in_km.isna().sum()
# dfo.dtypes
# dfo.shape[0] - pd.to_numeric(dfo.year, errors='coerce', dtype_backend = 'pyarrow').dropna(axis=0).shape[0]

In [ ]:
def check_non_numeric(s:pd.Series):
    # print (s.dtype)
    if 'pyarrow' in str(s.dtype):
        idx = pd.to_numeric(s, errors='coerce', dtype_backend='pyarrow').isna()
    else:
        idx = pd.to_numeric(s, errors='coerce').isna()
    return idx  # return index of non numeric


### Data Cleansing

In [ ]:
# drop non-numeric year values
df = dfo.copy()
# df = df[~check_non_numeric(df.year)].\
df = df[~((check_non_numeric(dfo.year)) | check_non_numeric(dfo.price_in_euro))].\
    assign(year = lambda x: x.year.astype(int))\
    .assign (price_in_euro = lambda x: x.price_in_euro.astype(float))
    # include only year values between (1900, 2023)
df = df.loc[df.year.between(1900, 2023)]

        
"number_of_dropped_rows:", dfo.shape[0]- df.shape[0]

### Plots

In [ ]:
df['year'].value_counts().sort_index().plot(ylabel="# of cars", xlabel='Year', title="Number of Cars by Manufacture Year")
plt.show()
df[['brand', 'year']].value_counts().head(20).plot(kind='barh', xlabel="# of cars", ylabel='Brand-Year', title="Top 20 Cars' brands-year")
plt.show()
df[['brand', 'model', 'year']].value_counts().head(20).plot(kind='barh', xlabel="# of cars", ylabel='Brand-Model-Year', title="Top 20 Cars' brand/model/year")
plt.show()
df[['brand', 'model', 'year', 'price_in_euro']].groupby(['brand', 'model', 'year']).mean()\
    .sort_values('price_in_euro', ascending=False).head(20)\
    .assign(price_in_euro = lambda x: x/1000000) \
    .plot(kind='barh', xlabel="Avg Price in Milion Euro", ylabel='Brand-Model-Year', title="Top 20 cars' prices") 
plt.show()

df[['brand','price_in_euro']].groupby(['brand']).mean()\
    .sort_values('price_in_euro', ascending=False).head(20)\
    .assign(price_in_euro = lambda x: x/1000) \
    .plot(kind='barh', xlabel="Avg Price in K Euro", ylabel='Brand', title="Top 20 cars' prices") 
plt.show()

In [ ]:
# Highest/Lowest average prices cars by brand-model-year
df[['brand', 'model', 'year', 'price_in_euro']].groupby(['brand', 'model', 'year']).mean().sort_values('price_in_euro', ascending=False)#.head(15)

In [ ]:
categ = pd.DataFrame()
categ = categ.assign (brand = df.brand.astype('category').cat.codes)\
        .assign(model = df.model.astype('category').cat.codes)\
        .assign(year = df.year.astype('category').cat.codes)\
        .assign(price_in_euro = df.price_in_euro)\
        .assign(brand_year = (df.brand+"-"+df.year.astype(str)))#.astype('category').cat.codes)


# df[['brand', 'year', 'price_in_euro']].corr(numeric_only=True)
    # .sort_values('price_in_euro', ascending=False).describe()
# x[['brand', 'price_in_euro']].corr()
categ

In [ ]:
categ[['brand_year', 'price_in_euro']].groupby('brand_year').agg(['mean', 'count'])#.head(20)#.plot(x='price_in_euro', y='price_in_euro')

### Working 

In [ ]:
# dfo.head()
# dfo.brand.value_counts()
# dfo[['year', 'registration_date']].head()
# df = dfo.assign(year_e = pd.to_numeric(dfo.year, errors='coerce'))
# df = dfo.assign(years_in_service = 2023-pd.to_numeric(dfo.year, errors='coerce'))
# dfo.year.astype(int, errors='ignore') - 2023
df[['brand', 'year', 'price_in_euro']].groupby(['brand', 'year']).mean()
# 2023 - dfo.year.astype(int, errors='ignore')
# df.head()
# df.years_on_service
# df.years_in_service = 2023 - df.years_in_service
# df = df.loc[df.years_in_service >= 0]
# df[['year', 'year_e']].value_counts()
# df[['year']].value_counts()


## Regression